libs

In [36]:
import pandas as pd
from  sklearn.feature_extraction.text import CountVectorizer
from unidecode import unidecode
from gensim.models import Word2Vec, KeyedVectors
from nltk.tokenize import word_tokenize
from string import punctuation
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier
import spacy
from gensim.models.callbacks import CallbackAny2Vec
import pickle


LOAD DATA

In [2]:
train_df = pd.read_csv('C:/Users/Usuario/Desktop/word_embeeding_data/treino.csv')
test_df = pd.read_csv('C:/Users/Usuario/Desktop/word_embeeding_data/teste.csv')

print(train_df.head())
print(test_df.head())

                                               title  \
0  Após polêmica, Marine Le Pen diz que abomina n...   
1  Macron e Le Pen vão ao 2º turno na França, em ...   
2  Apesar de larga vitória nas legislativas, Macr...   
3  Governo antecipa balanço, e Alckmin anuncia qu...   
4  Após queda em maio, a atividade econômica sobe...   

                                                text        date   category  \
0  A candidata da direita nacionalista à Presidên...  2017-04-28      mundo   
1  O centrista independente Emmanuel Macron e a d...  2017-04-23      mundo   
2  As eleições legislativas deste domingo (19) na...  2017-06-19      mundo   
3  O número de ocorrências de homicídios dolosos ...  2015-07-24  cotidiano   
4  A economia cresceu 0,25% no segundo trimestre,...  2017-08-17    mercado   

  subcategory                                               link  
0         NaN  http://www1.folha.uol.com.br/mundo/2017/04/187...  
1         NaN  http://www1.folha.uol.com.br/mundo/2017

LOAD MODEL

In [3]:
cbow_model = KeyedVectors.load_word2vec_format('model/cbow_model.txt')
sg_model = KeyedVectors.load_word2vec_format('model/sg_model.txt')

In [4]:
vector_size = cbow_model.vector_size
print(f"O modelo possui {vector_size} dimensões.")

O modelo possui 300 dimensões.


VETORIZACAO

In [17]:
def tokenizador(texto, nlp):
    doc = nlp(texto)
    lista = []
    for txt in doc:
        if not txt.is_stop and txt.is_alpha:
            lista.append(txt.text.lower())

    return lista

def combinar_vet_sum(tokens, model):
    result = np.zeros((1, model.vector_size))
    for token in tokens:
        try:
            result += model.get_vector(token)
        except KeyError:
            pass
    return result

def matriz_vetores(textos, model, nlp):
    matriz = np.zeros((len(textos), model.vector_size))
    for i, texto in enumerate(textos.copy()):
        if i % 5000 == 0:  
            print(f"{i}/{len(textos)}")
        tokens = tokenizador(texto, nlp)
        matriz[i] = combinar_vet_sum(tokens, model)
    return matriz

In [37]:
nlp = spacy.load('pt_core_news_sm', disable=['paser', 'ner', 'tagger', 'textcat'])

In [19]:
matriz_train_cbow = matriz_vetores(train_df['title'], cbow_model, nlp)
matriz_test_cbow = matriz_vetores(test_df['title'], cbow_model, nlp)

print(matriz_train_cbow.shape)
print(matriz_test_cbow.shape)

0/90000
5000/90000
10000/90000
15000/90000
20000/90000
25000/90000
30000/90000
35000/90000
40000/90000
45000/90000
50000/90000
55000/90000
60000/90000
65000/90000
70000/90000
75000/90000
80000/90000
85000/90000
0/20513
5000/20513
10000/20513
15000/20513
20000/20513
(90000, 300)
(20513, 300)


In [21]:
matriz_train_sg = matriz_vetores(train_df['title'], sg_model, nlp)
matriz_test_sg = matriz_vetores(test_df['title'], sg_model, nlp)

print(matriz_train_sg.shape)
print(matriz_test_sg.shape)

0/90000
5000/90000
10000/90000
15000/90000
20000/90000
25000/90000
30000/90000
35000/90000
40000/90000
45000/90000
50000/90000
55000/90000
60000/90000
65000/90000
70000/90000
75000/90000
80000/90000
85000/90000
0/20513
5000/20513
10000/20513
15000/20513
20000/20513
(90000, 300)
(20513, 300)


CLASSIFICADOR CBOW

In [28]:
lr_model_cbow = LogisticRegression(max_iter=1000)
lr_model_cbow.fit(matriz_train_cbow, train_df['category'])
print('modelo treinado')
print(lr_model_cbow.n_iter_)

modelo treinado
[343]


In [29]:
score_all_cbow = lr_model_cbow.score(matriz_test_cbow, test_df['category'])
print(score_all_cbow)

predictions_cbow = lr_model_cbow.predict(matriz_test_cbow)
classification_cbow = classification_report(test_df['category'], predictions_cbow)
print(classification_cbow)

0.7787256861502462


              precision    recall  f1-score   support

     colunas       0.81      0.71      0.75      6103
   cotidiano       0.62      0.79      0.69      1698
     esporte       0.91      0.86      0.89      4663
   ilustrada       0.13      0.82      0.22       131
     mercado       0.83      0.77      0.80      5867
       mundo       0.73      0.82      0.77      2051

    accuracy                           0.78     20513
   macro avg       0.67      0.80      0.69     20513
weighted avg       0.81      0.78      0.79     20513



In [24]:
dc_cbow = DummyClassifier()
dc_cbow.fit(matriz_train_cbow, train_df['category'])
dc_cbow_predict = dc_cbow.predict(matriz_test_cbow)

dc_cbow_classification = classification_report(test_df['category'], dc_cbow_predict)
print(dc_cbow_classification)

c:\Users\Usuario\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

     colunas       0.30      1.00      0.46      6103
   cotidiano       0.00      0.00      0.00      1698
     esporte       0.00      0.00      0.00      4663
   ilustrada       0.00      0.00      0.00       131
     mercado       0.00      0.00      0.00      5867
       mundo       0.00      0.00      0.00      2051

    accuracy                           0.30     20513
   macro avg       0.05      0.17      0.08     20513
weighted avg       0.09      0.30      0.14     20513



c:\Users\Usuario\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Usuario\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


CLASSIFICAÇÃO SG

In [25]:
lr_model_sg = LogisticRegression(max_iter=200)
lr_model_sg.fit(matriz_train_sg, train_df['category'])
print('modelo treinado')
print(lr_model_sg.n_iter_)

modelo treinado
[155]


In [26]:
score_all_sg = lr_model_sg.score(matriz_test_sg, test_df['category'])
print(score_all_sg)

predictions_sg = lr_model_sg.predict(matriz_test_sg)
classification_sg = classification_report(test_df['category'], predictions_sg)
print(classification_sg)

0.7969580266172671
              precision    recall  f1-score   support

     colunas       0.81      0.72      0.76      6103
   cotidiano       0.65      0.81      0.72      1698
     esporte       0.94      0.88      0.90      4663
   ilustrada       0.14      0.86      0.24       131
     mercado       0.84      0.80      0.82      5867
       mundo       0.76      0.84      0.80      2051

    accuracy                           0.80     20513
   macro avg       0.69      0.82      0.71     20513
weighted avg       0.82      0.80      0.81     20513



In [27]:
dc_sg = DummyClassifier()
dc_sg.fit(matriz_train_sg, train_df['category'])
dc_sg_predict = dc_sg.predict(matriz_test_sg)

dc_sg_classification = classification_report(test_df['category'], dc_sg_predict)
print(dc_sg_classification)

c:\Users\Usuario\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

     colunas       0.30      1.00      0.46      6103
   cotidiano       0.00      0.00      0.00      1698
     esporte       0.00      0.00      0.00      4663
   ilustrada       0.00      0.00      0.00       131
     mercado       0.00      0.00      0.00      5867
       mundo       0.00      0.00      0.00      2051

    accuracy                           0.30     20513
   macro avg       0.05      0.17      0.08     20513
weighted avg       0.09      0.30      0.14     20513



c:\Users\Usuario\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Usuario\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


SAVE

In [35]:
with open("model/lr_model_cbow.pkl", 'wb') as f:
    pickle.dump(lr_model_cbow, f)

with open('model/lr_model_sg.pkl', 'wb') as f:
    pickle.dump(lr_model_sg, f)

In [40]:
with open("model/nlp.pkl", 'wb') as f:
    pickle.dump(nlp, f)